In [1]:
import time
import  math
import numpy as np
from numpy.linalg import inv
import numpy.linalg as npl
import numpy.random as npr
import pandas as pd
import cufflinks as cf
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import linalg
from scipy.sparse import diags
from sympy import *
import sympy as sym
sym.init_printing()
from IPython.display import display, Math

In [2]:
from methods import jacobi,jacobi_num, sor, gauss_seidel, gauss_elimination, gaussElim
from check_matrix import characteristics, isDDM, create_block_triag

In [3]:
from scipy.sparse import random
from scipy.stats import rv_continuous
from numpy.random import default_rng
# random sparce matrix
class CustomDistribution(rv_continuous):
    def _rvs(self,  size=None, random_state=None):
        return random_state.standard_normal(size)
rng = default_rng()
X = CustomDistribution(seed=rng)
Y = X()  # get a frozen version of the distribution

In [4]:
def create_mat(n,add=4):
    #S = random(n, n, density=0.75, random_state=rng, data_rvs=Y.rvs)
    #A = S.A
    A=np.random.randint(low=-1,high=3,size=(n,n))
    for i in range(n):
        for j in range(n):
            if i == j:
                A[i][j]+= sum(abs(A[i]))+1

    A[0][0] += 0.5
    A[n-1][n-1] += 0.5
    return A

In [67]:
time_list = []
iter_list = []
mat, jac_it, gasei_it, sor_it, gael_it = [],[],[],[],[]
jac_time, gasei_time, sor_time, gael_time = [],[],[],[]


max_iter = 1000000
mat_gr = [80, 160, 320, 640, 1280]
rg = range(10,101,10)

In [68]:
for n in rg:
    A = create_block_triag(n)
    b = np.ones((n))

    if  characteristics(A) == 0:
        A = create_block_triag(n)
        print("Determinante ist Null")
        continue

    if isDDM(m=A,n=n):
        print(f"Lauf: {n}")

        true_sol = npl.solve(A, b)
        jac_erg, jacobi_time, k_jacobi = jacobi(A,b, iter=max_iter)
        gauss_seidel_erg, gauss_seidel_time, k_gauss_seidel = gauss_seidel(A, b, iter=max_iter)
        sor_erg, sor_t, k_sor = sor(A, b, w_in=None, iter=max_iter)
        gauss_elim_erg, gauss_elim_time,c = gaussElim(A=A,B=b)


        mat.append(n)
        jac_it.append(k_jacobi)
        gasei_it.append(k_gauss_seidel)
        sor_it.append(k_sor)
        gael_it.append(c)

        jac_time.append(jacobi_time)
        gasei_time.append(gauss_seidel_time)
        sor_time.append(sor_t)
        gael_time.append(gauss_elim_time)

        time_list.append([ jacobi_time, gauss_seidel_time, sor_t, gauss_elim_time ])
        iter_list.append([n,k_jacobi, k_gauss_seidel, k_sor])

Lauf: 10
Lauf: 20
Lauf: 30
Lauf: 40
Lauf: 50
Lauf: 60
Lauf: 70
Lauf: 80
Lauf: 90
Lauf: 100


In [69]:
#print(time_list)

In [70]:
mux = pd.MultiIndex.from_product([["Jacobi", "Gauss Seidel", "SOR", "Gauss Eliminierung"], ['Iterationen','Zeit (sek)']])
df = pd.DataFrame( columns=mux)


df["Jacobi","Iterationen"] = jac_it
df["Gauss Seidel","Iterationen"] = gasei_it
df["SOR","Iterationen"] = sor_it
df["Gauss Eliminierung","Iterationen"] = gael_it
df["Jacobi","Zeit (sek)"] = jacobi_time
df["Gauss Seidel","Zeit (sek)"] = gasei_time
df["SOR","Zeit (sek)"] = sor_time
df["Gauss Eliminierung","Zeit (sek)"] = gael_time
df["Matrixgröße"] = mat
df.set_index("Matrixgröße", inplace=True)
df

Jacobi            Gauss Seidel                    SOR  \
            Iterationen Zeit (sek)  Iterationen Zeit (sek) Iterationen   
Matrixgröße                                                              
10                  156    0.8906s           87    0.0000s          21   
20                  575    0.8906s          319    0.0156s          43   
30                 1255    0.8906s          696    0.0312s          65   
40                 2197    0.8906s         1217    0.1562s          88   
50                 3401    0.8906s         1884    0.2344s         112   
60                 4866    0.8906s         2695    0.3750s         136   
70                 6593    0.8906s         3651    0.5625s         160   
80                 8582    0.8906s         4752    0.2500s         185   
90                10832    0.8906s         5998    0.4219s         209   
100               13344    0.8906s         7389    0.6250s         234   

                       Gauss Eliminierung             
            Zeit (sek)        Iterationen Zeit (sek)  
Matrixgröße                                           
10             0.0000s                 64    0.0000s  
20             0.0000s                229    0.0000s  
30             0.0156s                494    0.0000s  
40             0.1250s                859    0.0000s  
50             0.3125s               1324    0.0625s  
60             0.5312s               1889    0.0000s  
70             0.5469s               2554    0.0000s  
80             0.2500s               3319    0.0000s  
90             0.5000s               4184    0.0000s  
100            0.6250s               5149    0.0000s

In [12]:
def add_top_column(df, top_col, inplace=False):
    if not inplace:
        df = df.copy()

    df.columns = pd.MultiIndex.from_product([[top_col], df.columns])
    return df


orig_df = pd.DataFrame([[1, 2], [3, 4]], columns=['Iterationen', 'Zeit (sek)'])
new_df = add_top_column(orig_df, "Jacobi")
new_df2 = add_top_column(orig_df, "Gauss Seidel")
new_df3 = add_top_column(orig_df, "SOR")
new_df4 = add_top_column(orig_df, "Gauss Eliminierung")
res_df = pd.concat([new_df, new_df2, new_df3, new_df4], axis=1)
res_df

Jacobi            Gauss Seidel                    SOR             \
  Iterationen Zeit (sek)  Iterationen Zeit (sek) Iterationen Zeit (sek)   
0           1          2            1          2           1          2   
1           3          4            3          4           3          4   

  Gauss Eliminierung             
         Iterationen Zeit (sek)  
0                  1          2  
1                  3          4

In [ ]:
n=20
A = create_block_triag(n)
b = np.ones((n))
true_sol = npl.solve(A, b)

jac_erg, jacobi_time, k_jacobi = jacobi(A,b, iter=max_iter)
gauss_seidel_erg, gauss_seidel_time, k_gauss_seidel = gauss_seidel(A, b, iter=max_iter)
sor_erg, sor_time, k_sor = sor(A, b, w_in=None, iter=max_iter)
gauss_elim_erg, gauss_elim_time,c = gaussElim(A,b)

#jac_erg.append(true_sol)
#gauss_seidel_erg.append(true_sol)
#sor_erg.append(true_sol)

time_df = pd.DataFrame(columns=["Jacobi", "Gauss Seidel", "SOR", "Gauss Eliminierung"])
time_df.loc[0]=[ jacobi_time, gauss_seidel_time, sor_time, gauss_elim_time ]
time_df.loc[1]=[ int(k_jacobi), k_gauss_seidel, k_sor, c]
time_df.index = ["Zeit (sek)", "Iterrationen"]

time_df.loc["Iterrationen"] = time_df.loc["Iterrationen"].astype(float).astype(int)
time_df

In [ ]:
mux = pd.MultiIndex.from_product([["Jacobi", "Gauss Seidel", "SOR", "Gauss Eliminierung"], ['Itterationen','Zeit (sek)']])
df = pd.DataFrame( columns=mux)

df

In [ ]:
gs_fehler=np.round(gauss_seidel_erg[len(gauss_seidel_erg)-1]-true_sol,4)
jk_fehler=np.round(jac_erg[len(jac_erg)-1]-true_sol,4)
sor_fehler=np.round(sor_erg[len(sor_erg)-1]-true_sol,4)

jr = Matrix(jk_fehler)
gr = Matrix(gs_fehler)
sr = Matrix(sor_fehler)
#display(Math('\ \\text{ Jacobi : }%s   \\text{Gauss-Seidel :} %s \\text{SOR :} %s' % (sym.latex(sym.simplify(jr)) , (sym.latex(sym.simplify(gr))), (sym.latex(sym.simplify(sr)))  ))   )

In [ ]:
latex_name =f"time_table_m{n}.tex"
#time_df.to_latex("C:\\Users\\eaut2\\DataspellProjects\\Numerik-Projekt\\Test\\"+f"{latex_name}", index=True, bold_rows=True, caption="Beispieltext", position="h!", label="example")

In [ ]:
l1, l2, l3,l4, la = symbols("x1 x2 x3 x.. x")
l = Matrix([l1,l2,l3,l4])
r = Matrix(b)
a = Matrix(A)
#display(Math('\ \\text{  }%s   \\text{*} %s \\text{= } %s' % (sym.latex(sym.simplify(a)) , (sym.latex(sym.simplify(l))), (sym.latex(sym.simplify(r)))  ))   )

In [ ]:
dict = {'Jacobi Verfahren': jac_erg,
        'Gauss-Seidel Verfahren': gauss_seidel_erg,
        "SOR Verfahren":sor_erg
        }
#label = [f'x{i}' for i in range(0,n)]
label = [f'x0 - x{n-1} ']

for verfahren_name,data in dict.items():

    plt.figure(figsize=(9,6))
    plt.plot(data) # linestyle='',marker='.'
    #plt.axvline(x=k_sor+0.5,linestyle='--',c='red')
    plt.title(verfahren_name, fontweight='bold', fontsize=17, pad=20)
    plt.xticks(rotation='0', fontsize=14)
    plt.yticks(rotation='0', fontsize=14)
    plt.xlabel('Iteration', fontsize=14, labelpad=15, weight ='bold')
    plt.ylabel('Wert', fontsize=14, labelpad=15, weight ='bold')
    plt.legend(labels = label,frameon=True,loc='best', shadow=True, fontsize="large")
    plt.tight_layout(h_pad=3.0)
    plt.grid(True)
    plt.show()

Ideen: LGS von n = 2 bis n = 6 mio

Plot: Fehler, Wert,